# Pyomo Summer Workshop 2018 Exercise Problem - Pyomo Nonlinear Problems

This notebook contains material from the Pyomo Summer Workshop 2018 by the Pyomo Developement Team and the notebook is developed by David Bernal (dbernaln at purdue.edu), Zedong Peng (peng_zedong at 123.com), and Albert Lee (lee4382 at purdue.edu);  the content is available on **[Github](https://github.com/AlbertLee125/pyomo-summer-ws)**.
The original sources of this material are available on **[Pyomo](http://www.pyomo.org/workshop-examples)**.

In [1]:
import pyomo.environ as pyo

## Example: Rosenbrock function
The Rosenbrock function is defined as below:

$$
f(x,y) = (1-x)^2 + 100(y-x^2)^2
$$

Minimize the Rosenbrock function using Pyomo and IPOPT.
Initialize $x$ and $y$ to 1.5 and 1.5, respectively.

In [2]:
model = pyo.ConcreteModel()
model.x = pyo.Var(initialize=1.5)
model.y = pyo.Var(initialize=1.5)

def rosenbrock(model):
    return (1.0-model.x)**2 \
        + 100.0*(model.y - model.x**2)**2
model.obj = pyo.Objective(rule=rosenbrock, sense=pyo.minimize)

pyo.SolverFactory('ipopt').solve(model)
model.pprint()

ipopt


ApplicationError: No executable found for solver 'ipopt'

## Exercise 1: Nonlinear Problems
### 1.1. Alternative Initialization
Effective initialization can be critical for solving nonlinear problems, since they can have several local solutions and numerical difficulties.
Solve the Rosenbrock example using different initial values for the $x$ variables.
Write a loop that varies the initial value from $2.0$ to $6.0$, solves the problem, and prints the solution for each iteration of the loop.
(A soution for this problem can be found in `rosenbrock.init.sol.py`.)

In [ ]:
model = pyo.ConcreteModel()
model.x = pyo.Var()
model.y = pyo.Var()

def rosenbrock(m):
    return (1.0-m.x)**2 + 100.0*(m.y - m.x**2)**2
model.obj = pyo.Objective(rule=rosenbrock, sense=pyo.minimize)


solver = pyo.SolverFactory('ipopt')

print('x_init, y_init, x_soln, y_soln')
y_init = 5.0
for x_init in range(2, 6):
    model.x = x_init
    model.y = 5.0

    solver.solve(model)

    print("{0:6.2f}  {1:6.2f}  {2:6.2f}  {3:6.2f}".format(x_init, \
            y_init, pyo.value(model.x), pyo.value(model.y)))

### 1.2. Evaluation errors
Consider the following problem with initial values $x=5$, $y=5$.

$$
\begin{aligned}
\min_{x,y} &~ f(x,y) = (x - 1.01)^2 + y^2 \\
\text{s.t.} &~~ y = \sqrt{x - 1.0}
\end{aligned}
$$

1. Starting with `evalueation_error_incomplete.py`, formulate this Pyomo model and solve using IPOPT.
You should get list of errors from the solver.
Add the IPOPT solver options `solver.options[`halt_on_ampl_error`]=`yes`` to find the problem.
(Hint: error output might be ordered strangely, look up in the console output.)
What did you discover?
How might you fix this?
(Solution for this can be found in `evaluation_error_bounds_soln.py`)

```python
# evaluation_error_incomplete.py
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == sqrt(m.x - 1.0)
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
# TODO: ADD SOLVER OPTIONS HERE
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )
```

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == pyo.sqrt(m.x - 1.0)
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

2. Add bounds $x \geq 1$ to fix this problem.
Resolve the problem.
Comment on the number of iterations and the quality of the solution.
(Note: The problem still occurs because $x \geq 1$ is not enforced exactly, and small numerical values still cause the error.)
(A soution for this can be found in `evaluation_error_bounds_soln.py`.)

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0, bounds=(1,None))
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == sqrt(m.x - 1.0)
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

3. Think about other solutions for this problem.
(e.g., $ x \geq 1.001$).
(A solution for this can be found in `evaluation_error_bounds2_soln.py`.)

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0, bounds=(1.001,None))
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == pyo.sqrt(m.x - 1.0)
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.options['halt_on_ampl_error'] = 'yes'
solver.solve(model, tee=True)

print( pyo.value(model.x) )
print( pyo.value(model.y) )

### 1.3. Alternative Formulations
Consider the following problem with initial values $x=5$ and $y=5$.

$$
\begin{aligned}
\min_{x,y} &~ f(x,y) = (x - 1.01)^2 + y^2 \\
\text{s.t.} &~~ \frac{x-1}{y} = 1
\end{aligned}
$$

Note that the solution to this problem is $x=1.005$ and $y=0.005$.
There are several ways that the problem above can be reformulated.
Some examples are shown below.
Which ones do you expect to be better?
Why?
Starting with `formulation_incomplete.py` finish the Pyomo model for each of the iterations and quality of solutions.
What can you learn about problem formulation from this examples?

```python
# formulation_incomplete.py
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    # TODO: SPECIFY CONSTRAINT HERE
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)

print(pyo.value(model.x))
print(pyo.value(model.y))
```

1. (A solution can be found in `formulation_1_soln.py`.)
$$
\begin{aligned}
\min_{x,y} &~ f(x,y) = (x - 1.01)^2 + y^2 \\
\text{s.t.} &~~ \frac{x-1}{y} = 1
\end{aligned}
$$

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return (m.x - 1.0) / m.y == 1.0
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)

print(pyo.value(model.x))
print(pyo.value(model.y))

2. (A solution can be found in `formulation_2_soln.py`.)

$$
\begin{aligned}
\min_{x,y} &~ f(x,y) = (x - 1.01)^2 + y^2 \\
\text{s.t.} &~~ \frac{x}{y+1} = 1
\end{aligned}
$$

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.x / (m.y + 1.0) == 1.0
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)

print(pyo.value(model.x))
print(pyo.value(model.y))


3. (A solution can be found in `formulation_3_soln.py`.)

$$
\begin{aligned}
\min_{x,y} &~ f(x,y) = (x - 1.01)^2 + y^2 \\
\text{s.t.} &~~ y = x-1 \\
\end{aligned}
$$

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0)

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return m.y == m.x - 1.0
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)

print(pyo.value(model.x))
print(pyo.value(model.y))

4. Bounds and initialization can be very helpful when solving nonlinear optimization problems.
Starting with `formulation_incomplete.py`, resolver the original problem, but add bounds, $y \geq 0$. 
Note the number of iterations and quality of solution, and compare with what you found in 1.2.2.
(A solution can be found in `formulation_4_soln.py`.)

In [ ]:
model = pyo.ConcreteModel()

model.x = pyo.Var(initialize=5.0)
model.y = pyo.Var(initialize=5.0, bounds=(0,None))

def obj_rule(m):
    return (m.x-1.01)**2 + m.y**2
model.obj = pyo.Objective(rule=obj_rule)

def con_rule(m):
    return (m.x - 1.0) / m.y == 1.0
model.con = pyo.Constraint(rule=con_rule)

solver = pyo.SolverFactory('ipopt')
solver.solve(model, tee=True)

print(pyo.value(model.x))
print(pyo.value(model.y))

### 1.4. Reactor design problem (Hart et al., 2017; Bequette, 2003)
In this example, we will consider a chemical reactor designed to produce product $B$ from reactant $A$ using a reaction scheme known as the Van de Vusse reaction:

$$
A \stackrel{k_1}{\rightarrow} B \stackrel{k_2}{\rightarrow} C \\
2A \stackrel{k_3}{\rightarrow} D
$$

Under the appropriate assumptions, $F$ is the volumetric flowrate through the tank.
The concentration of component $A$ in the feed is $c_{Af}$, and the concentrations in the reactor are equivalent to the concentrations of each component flowing out of the reactor, given by $c_A$, $c_B$, $c_C$, and $c_D$.

If the reactor is too small, we will not produce sufficient quantity of $B$, and if the reactor is too large, much of $B$ will be further reacted to form the undesired product $C$.
Therefore, our goal is to solve for the reactor volume that maximizes the outlet concentration for product $B$.

The steady-state mole balances for each of the four components are given by,

$$
\begin{aligned}
0 =& \frac{F}{V} c_{Af} - \frac{F}{V} c_{A} - k_1 c_A - 2 k_3 c_A^2 \\
0 =& - \frac{F}{V} c_{B} + k_1 c_A - k_2 c_B \\
0 =& - \frac{F}{V} c_{C} + k_2 c_B \\
0 =& - \frac{F}{V} c_{D} + k_3 c_A^2
\end{aligned}
$$

The known parameters for the system are,

$$
c_{Af} = 10 \frac{\text{gmol}}{\text{m}^3} \quad k_1 = \frac{5}{6} \text{min}^{-1} \quad k_2 = \frac{5}{6} \text{min}^{-1} \quad k_3 = \frac{1}{6000} \frac{\text{m}^3}{\text{mol} \cdot \text{min}}.
$$

Formulate and solve this optimization problem using Pyomo.
Since the volumetric flowrate $F$ always appears as the numerator over the reactor volume $V$, it is common to consider this ratio as a single variable, called the space-velocity $SV$.
(A solution to this problem can be found in `reactor_design_soln.py`.)

In [ ]:
# create the concrete model
model = pyo.ConcreteModel()

# set the data (native python data)
k1 = 5.0/6.0     # min^-1
k2 = 5.0/3.0     # min^-1
k3 = 1.0/6000.0  # m^3/(gmol min)
caf = 10000.0    # gmol/m^3

# create the variables
model.sv = pyo.Var(initialize = 1.0, within=pyo.PositiveReals)
model.ca = pyo.Var(initialize = 5000.0, within=pyo.PositiveReals)
model.cb = pyo.Var(initialize = 2000.0, within=pyo.PositiveReals)
model.cc = pyo.Var(initialize = 2000.0, within=pyo.PositiveReals)
model.cd = pyo.Var(initialize = 1000.0, within=pyo.PositiveReals)

# create the objective
model.obj = pyo.Objective(expr = model.cb, sense=maximize)

# create the constraints
model.ca_bal = pyo.Constraint(expr = (0 == model.sv * caf \
                 - model.sv * model.ca - k1 * model.ca \
                 -  2.0 * k3 * model.ca ** 2.0))

model.cb_bal = pyo.Constraint(expr=(0 == -model.sv * model.cb \
                 + k1 * model.ca - k2 * model.cb))

model.cc_bal = pyo.Constraint(expr=(0 == -model.sv * model.cc \
                 + k2 * model.cb))

model.cd_bal = pyo.Constraint(expr=(0 == -model.sv * model.cd \
                 + k3 * model.ca ** 2.0))

pyo.SolverFactory('ipopt').solve(model)
model.pprint()
